In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DATA_DIR = Path(os.path.abspath('')).resolve() / "data"
STRATS_PATH     = DATA_DIR / "strategies" / "strategy_returns.parquet"
COMMON_SIG_PATH = DATA_DIR / "signals" / "common_vol_signal.parquet"
ONEFAC_SIG_PATH = DATA_DIR / "signals" / "onefactor_signals.parquet"

EVAL_DIR  = DATA_DIR / "eval"
PLOTS_DIR = EVAL_DIR / "plots"
YEAR = 2025
MONTHS_PER_YEAR = 12

In [ ]:
# Some util functions

def ensure_dirs():
    """
    Create the directories to store the info
    """
    EVAL_DIR.mkdir(parents=True, exist_ok=True)
    PLOTS_DIR.mkdir(parents=True, exist_ok=True)

def ytd_mask(index, year):
    idx = pd.to_datetime(index)
    return (idx >= pd.Timestamp(f"{year}-01-01")) & (idx <= idx.max())

def cagr_from_monthlies(r: pd.Series) -> float:
    """Annualize geometric return from monthly simple returns."""
    r = r.dropna()
    n = len(r)
    if n == 0:
        return np.nan
    g = (1.0 + r).prod() - 1.0
    return (1.0 + g) ** (MONTHS_PER_YEAR / n) - 1.0

def ann_vol_from_monthlies(r: pd.Series) -> float:
    """Annualized volatility from monthly simple returns (sample std)."""
    r = r.dropna()
    if len(r) < 2:
        return np.nan
    return r.std(ddof=1) * np.sqrt(MONTHS_PER_YEAR)

def sharpe_from_monthlies(total: pd.Series, rf: pd.Series) -> float:
    """
    Annualized Sharpe using monthly excess returns:
        Sharpe = (12 * mean(excess)) / (sqrt(12) * std(excess))
    """
    excess = (total - rf).dropna()
    if len(excess) < 2:
        return np.nan
    mu_ann = excess.mean() * MONTHS_PER_YEAR
    vol_ann = excess.std(ddof=1) * np.sqrt(MONTHS_PER_YEAR)
    return mu_ann / vol_ann if vol_ann > 0 else np.nan

def max_drawdown(total: pd.Series) -> float:
    """Max drawdown on the cumulative wealth path from monthly total returns."""
    wealth = (1.0 + total.fillna(0.0)).cumprod()
    peak = wealth.cummax()
    dd = wealth / peak - 1.0
    return float(dd.min()) if len(dd) else np.nan

def summarize_metrics(total: pd.Series, rf: pd.Series) -> pd.Series:
    return pd.Series({
        "CAGR": cagr_from_monthlies(total),
        "AnnVol": ann_vol_from_monthlies(total),
        "Sharpe": sharpe_from_monthlies(total, rf),
        "MaxDrawdown": max_drawdown(total),
        "Months": total.dropna().shape[0],
    })

def plot_cumulative_returns(ytd_df: pd.DataFrame, outpath: Path):
    """Line chart of cumulative total returns (wealth index)."""
    wealth = (1.0 + ytd_df).fillna(0.0).cumprod()
    ax = wealth.plot(figsize=(9, 5), linewidth=2)
    ax.set_title(f"Cumulative Total Return — {YEAR} YTD")
    ax.set_ylabel("Growth of $1")
    ax.set_xlabel("Month")
    ax.grid(True, alpha=0.3)
    fig = ax.get_figure()
    fig.tight_layout()
    fig.savefig(outpath, dpi=144)
    plt.close(fig)

def plot_common_exposure(ytd_exposure: pd.Series, outpath: Path):
    """Line chart of lagged common exposure used each month."""
    ax = ytd_exposure.plot(figsize=(9, 3.8), linewidth=2)
    ax.set_title(f"Common-Vol Managed Exposure (lagged) — {YEAR} YTD")
    ax.set_ylabel("Exposure (x)")
    ax.set_xlabel("Month")
    ax.grid(True, alpha=0.3)
    fig = ax.get_figure()
    fig.tight_layout()
    fig.savefig(outpath, dpi=144)
    plt.close(fig)

def plot_onefactor_band(ytd_exposure_lag_df: pd.DataFrame, outpath: Path):
    """
    Plot per-asset lagged exposures as a band: min/max shaded with mean line.
    """
    exp_df = ytd_exposure_lag_df.dropna(how="all")
    if exp_df.empty:
        return
    exp_min = exp_df.min(axis=1)
    exp_max = exp_df.max(axis=1)
    exp_mean = exp_df.mean(axis=1)

    fig, ax = plt.subplots(figsize=(9, 3.8))
    ax.fill_between(exp_min.index, exp_min.values, exp_max.values, alpha=0.2, label="Min–Max range")
    exp_mean.plot(ax=ax, linewidth=2, label="Cross-sectional mean")
    ax.set_title(f"One-Factor Per-Asset Exposures (lagged) — {YEAR} YTD")
    ax.set_ylabel("Exposure (x)")
    ax.set_xlabel("Month")
    ax.grid(True, alpha=0.3)
    ax.legend()
    fig.tight_layout()
    fig.savefig(outpath, dpi=144)
    plt.close(fig)


In [4]:
ensure_dirs() # create the directories needed

strat = pd.read_parquet(STRATS_PATH)
common = pd.read_parquet(COMMON_SIG_PATH)
onefac = pd.read_parquet(ONEFAC_SIG_PATH)

# Align to monthly end
for df in (strat, common, onefac):
    df.index = pd.to_datetime(df.index)

# ----- Build the YTD slices for the testing period -----
strat.index = pd.to_datetime(strat.index)
common.index = pd.to_datetime(common.index)
onefac.index = pd.to_datetime(onefac.index)

# 2025 months that exist in strat (metrics don't require signals)
ytd_idx_strat = strat.index[(strat.index >= pd.Timestamp(f"{YEAR}-01-01")) &
                            (strat.index <= strat.index.max())]

####

# Average applied exposures in 2025
avg_x_common_2025 = common.reindex(ytd_idx_strat)["exposure_common_lag"].mean()
print(f"Avg common exposure (2025): {avg_x_common_2025:0.3f}")

if isinstance(onefac.columns, pd.MultiIndex):
    avg_x_one_2025 = onefac.reindex(ytd_idx_strat)["exposure_i_lag"].mean().mean()
else:
    avg_x_one_2025 = onefac.filter(like="exposure_i_lag").reindex(ytd_idx_strat).mean().mean()
print(f"Avg one-factor exposure (2025, cross-asset mean): {avg_x_one_2025:0.3f}")

####

ytd_rf  = strat.loc[ytd_idx_strat, "rf"]
ytd_ew  = strat.loc[ytd_idx_strat, "ew"]
ytd_com = strat.loc[ytd_idx_strat, "common_total"]
ytd_one = strat.loc[ytd_idx_strat, "onefactor_total"]

# ----- Exposures for plots (align to the same months if available) -----
# Common exposure (lagged)
ytd_x_com = common.reindex(ytd_idx_strat)["exposure_common_lag"].dropna()

# One-factor per-asset exposure band (lagged)
if isinstance(onefac.columns, pd.MultiIndex):
    ytd_x_one_df = onefac.reindex(ytd_idx_strat)["exposure_i_lag"].dropna(how="all")
else:
    ytd_x_one_df = onefac.filter(like="exposure_i_lag").reindex(ytd_idx_strat).dropna(how="all")


# ----- Metrics -----
rows = []
rows.append(("EW",        summarize_metrics(ytd_ew,  ytd_rf)))
rows.append(("CommonVol", summarize_metrics(ytd_com, ytd_rf)))
rows.append(("OneFactor", summarize_metrics(ytd_one, ytd_rf)))
metrics = pd.DataFrame({name: s for name, s in rows}).T
metrics = metrics[["Months", "CAGR", "AnnVol", "Sharpe", "MaxDrawdown"]]

# Save metrics
metrics.to_parquet(EVAL_DIR / "2025_metrics.parquet")
metrics.to_csv(EVAL_DIR / "2025_metrics.csv", float_format="%.6f")

# ----- Plots -----
rtns_df = pd.DataFrame({
    "EW": ytd_ew,
    "CommonVol": ytd_com,
    "OneFactor": ytd_one,
}).dropna(how="all")

if not rtns_df.empty:
    plot_cumulative_returns(rtns_df, PLOTS_DIR / "returns_cumulative_2025.png")

if not ytd_x_com.empty:
    plot_common_exposure(ytd_x_com, PLOTS_DIR / "exposure_common_2025.png")

if isinstance(ytd_x_one_df, pd.DataFrame) and not ytd_x_one_df.empty:
    plot_onefactor_band(ytd_x_one_df, PLOTS_DIR / "exposure_onefactor_band_2025.png")

# ----- Console preview -----
print("\n2025 YTD Metrics (monthly to date):")
print(metrics.to_string(float_format=lambda x: f"{x:0.4f}"))
print("\nSaved:")
print(f" - {Path(*EVAL_DIR.parts[EVAL_DIR.parts.index('data'):])/'2025_metrics.parquet'} & .csv")
print(f" - {Path(*PLOTS_DIR.parts[PLOTS_DIR.parts.index('data'):])/'returns_cumulative_2025.png'}")
print(f" - {Path(*PLOTS_DIR.parts[PLOTS_DIR.parts.index('data'):])/'exposure_common_2025.png'}")
print(f" - {Path(*PLOTS_DIR.parts[PLOTS_DIR.parts.index('data'):])/'exposure_onefactor_band_2025.png'}")

Avg common exposure (2025): 0.905
Avg one-factor exposure (2025, cross-asset mean): 0.910

2025 YTD Metrics (monthly to date):
           Months   CAGR  AnnVol  Sharpe  MaxDrawdown
EW         8.0000 0.1710  0.2658  0.5586      -0.1732
CommonVol  8.0000 0.1072  0.1787  0.4195      -0.1213
OneFactor  8.0000 0.0811  0.2348  0.2601      -0.1627

Saved:
 - data\eval\2025_metrics.parquet & .csv
 - data\eval\plots\returns_cumulative_2025.png
 - data\eval\plots\exposure_common_2025.png
 - data\eval\plots\exposure_onefactor_band_2025.png


c:\Users\yc_ya\anaconda3\envs\sep\lib\site-packages\pandas\plotting\_matplotlib\core.py:975: UserWarning: This axis already has a converter set and is updating to a potentially incompatible converter
  return ax.plot(*args, **kwds)
